In [1]:
import numpy as np

In [2]:
metrics = ['clr', 'manhattan', 'euclidean', 'cosine'] + [f'minkowski_{str(p)}' for p in np.array([0.5, 1, 2, 3, 4, 5])]
metrics

['clr',
 'manhattan',
 'euclidean',
 'cosine',
 'minkowski_0.5',
 'minkowski_1.0',
 'minkowski_2.0',
 'minkowski_3.0',
 'minkowski_4.0',
 'minkowski_5.0']

In [3]:
len(metrics)

10

In [ ]:
nns = np.arange(2, 13, 1)
nns

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [ ]:
for metric in metrics:
    for nn in nns:

        sbatch_str = f'''#!/bin/bash
#SBATCH --job-name=run_scan_{metric}_{nn}
#SBATCH --account=pi-apturkew
#SBATCH --partition=broadwl
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=20
#SBATCH --mem-per-cpu=2000
##SBATCH --output=myjob_%j.out      # Standard output log
##SBATCH --error=myjob_%j.err       # Standard error log
#SBATCH --mail-type=ALL
#SBATCH --mail-user=bertagna@rcc.uchicago.edu

source activate scan.env

cmd="python3 fast_scan_modular_metrics_1m_1nn.py {metric} {nn}"

echo $cmd

eval $cmd
'''
        with open(f'./{metric}_metric_{nn}nn.sbatch', 'w') as f:
            f.write(sbatch_str)

In [ ]:
for metric in metrics:
    sbatch_str = f'''#!/bin/bash
#SBATCH --job-name=run_scan_{metric}
#SBATCH --account=pi-apturkew
#SBATCH --partition=broadwl
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=20
#SBATCH --mem-per-cpu=2000
##SBATCH --output=myjob_%j.out      # Standard output log
##SBATCH --error=myjob_%j.err       # Standard error log
#SBATCH --mail-type=ALL
#SBATCH --mail-user=bertagna@rcc.uchicago.edu

source activate scan.env

cmd="python3 fast_scan_modular_metrics.py {metric}"

echo $cmd

eval $cmd
'''
    with open(f'./{metric}_metric.sbatch', 'w') as f:
        f.write(sbatch_str)

### CMD NOTES

In [ ]:
'''
put /Users/michaelbertagna/git/TGNE-2022/TGNE/clustering_optimization/clr_network_for_distances_*.csv.gz TGNE/clustering_optimization/

put /Users/michaelbertagna/git/TGNE-2022/TGNE/microarray_probe_alignment_and_filtering/allgood_filt_agg_tidy_2021aligned_qc_rma_expression_full.csv TGNE/microarray_probe_alignment_and_filtering/allgood_filt_agg_tidy_2021aligned_qc_rma_expression_full.csv

put /Users/michaelbertagna/git/TGNE-2022/TGNE/eggnog/complete_eggnog_annotation.csv TGNE/eggnog/complete_eggnog_annotation.csv

put /Users/michaelbertagna/git/TGNE-2022/active_fastas/*.csv active_fastas/
'''

In [1]:
'''
submit *_metric_{6..12}nn.sbatch -x clr* cosine*
'''

'\nsubmit *_metric_{6..12}nn.sbatch -x clr* cosine*\n'

In [ ]:
'''
submit minkowski_metric_1.0_11nn.sbatch
submit minkowski_metric_1.0_10nn.sbatch
submit manhattan_metric_10nn.sbatch
submit manhattan_metric_11nn.sbatch
submit clr_metric_4nn.sbatch
submit clr_metric_8nn.sbatch
submit clr_metric_5nn.sbatch
'''